In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import random
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
filenames = os.listdir("../input/train/train")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)
        
        
df = pd.DataFrame({'filename':filenames,
                  'category':categories})

df.head()

In [ ]:
filenames = os.listdir("../input/train/train")
sample = random.choice(filenames)
image = load_img("../input/train/train/"+sample)
plt.imshow(image)

In [ ]:
df['category'].value_counts().plot.bar()

In [ ]:
train_df, validation_df = train_test_split(df,test_size = 0.20, random_state = 42)

train_df = train_df.reset_index(drop=True)
validation_df = validation_df.reset_index(drop=True)

total_train = train_df.shape[0]
total_validate = validation_df.shape[0]
batch_size=15

In [ ]:
from keras.models import Sequential,model_from_json
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

image_width = 50
image_height = 50
image_size = (image_width, image_height)
image_channel = 3

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (image_width, image_height,image_channel), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3,3),activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
#Fitting the CNN into images
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)


train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "../input/train/train/", 
    x_col='filename',
    y_col='category',
    target_size=image_size,
    class_mode='binary',
    batch_size=batch_size
)



In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validation_df, 
    "../input/train/train/", 
    x_col='filename',
    y_col='category',
    target_size=image_size,
    class_mode='binary',
    batch_size=batch_size
)


In [ ]:
x, y = train_generator.next()
for i in range(0, 1):
    random_image = x[i]
    plt.imshow(random_image)
    plt.show()

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

callbacks = [earlystop, learning_rate_reduction]

In [ ]:
history = model.fit_generator(
    train_generator, 
    epochs= 25,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

In [ ]:
model.save_weights("model.h5")